In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from numba import njit
from datetime import datetime, timedelta
from scipy.spatial import Delaunay, ConvexHull

In [2]:
def parse_line(line):
    num_chars_id = 11
    num_chars_year = 4
    num_chars_month = 2
    num_chars_measure = 4
    num_chars_val = 5
    num_chars_bs = 3
    # header part
    id_val = line[:num_chars_id]
    block = line[num_chars_id:]
    year = int(block[:num_chars_year])
    block = block[num_chars_year:]
    month = int(block[:num_chars_month])
    block = block[num_chars_month:]
    measure = block[:num_chars_measure]
    block = block[num_chars_measure:]
    values = []
    
    while(len(block) > 0):
        if(len(values) == 31):
            break
        val = int(block[:num_chars_val])
        if(val == -9999):
            val = np.nan
        block = block[num_chars_val+num_chars_bs:]
        values.append(val)
    return id_val, year, month, measure, values

In [3]:
stations = pd.read_csv('ghcnd-stations.txt', sep=r'\s+', on_bad_lines='skip', header=None).loc[:,[0,1,2]].rename({0:'station', 1:'lat', 2:'lon'}, axis=1)
stations = stations[(stations.lon > -10)*(stations.lon < 40)*(stations.lat > 30)*(stations.lat < 70)].reset_index(drop=True)
stations = stations.set_index('station')

In [4]:
source_dir = 'ghcnd_all/ghcnd_all/'

In [ ]:
countries = ['IT', 'FR', 'GM', 'SP']
data_dict = {'station' : [], 'year' : [], 'month' : [], 'day' : [], 'measure' : [], 'value' : []}
min_year = 1980 
for station_name, row in stations.iterrows():
    if(not station_name[:2] in countries):
        continue
    #print(station_name)
    try:
        with open(source_dir + station_name + '.dly') as file:
            #lines = [line.replace('-9999', ' -9999') for line in file.readlines()]
            lines = file.readlines()
            for line in lines:
                id_val, year,  month, measure, values = parse_line(line)
                if(year < min_year):
                    continue
                num_values = len(values)
                data_dict['station'].extend([station_name]*num_values)
                data_dict['year'].extend([year]*num_values)
                data_dict['month'].extend([month]*num_values)
                data_dict['day'].extend(list(range(1, 1+ num_values)))
                data_dict['measure'].extend([measure]*num_values)
                data_dict['value'].extend(values)
    except Exception as e:
        print(e)

In [6]:
data = pd.DataFrame.from_dict(data_dict).sort_values(by=['year', 'month', 'day'])
data.dropna(axis=0, inplace=True)
data['date'] = pd.to_datetime(data[['year', 'month', 'day']])
data.drop(['year', 'month', 'day'], axis=1, inplace=True)
data.set_index(['station', 'date', 'measure'], inplace=True, drop=True)

In [42]:
measure_x = 'TAVG'
measure_y = 'PRCP'
subset = pd.merge(data.xs(measure_x, level='measure'), data.xs(measure_y, level='measure'),left_index=True, right_index=True, how='inner')

In [ ]:
spatial_data = []
for station, group in subset.groupby('station',):
    group_indexed = group.reset_index(level='station', drop=True)
    rolling_cov = group_indexed['value_x'].rolling('10D').cov(group_indexed['value_y'])
    rolling_varx = group_indexed['value_x'].rolling('10D').var()
    rolling_vary = group_indexed['value_y'].rolling('10D').var()
    mutual = np.log(rolling_varx*rolling_vary/(rolling_varx*rolling_vary-rolling_cov*rolling_cov))/2
    variance_mutual = rolling_cov*rolling_cov/(rolling_varx*rolling_vary)
    info = stations.loc[station]
    lat, lon = info['lat'], info['lon']
    spatial_data.append((lon, lat, (variance_mutual/mutual).dropna(axis=0).median()))
spatial_data = np.array(spatial_data)

In [54]:
%matplotlib qt
plt.scatter(spatial_data[:, 0], spatial_data[:, 1], c=spatial_data[:,2])
plt.colorbar()
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
spatial_data = []
for station, group in subset.groupby('station'):
    info = stations.loc[station]
    lat, lon = info['lat'], info['lon']
    cmatr = np.corrcoef(group['value_x'], group['value_y'])
    spatial_data.append((lon, lat, cmatr[0,1]))
spatial_data = np.array(spatial_data)

In [33]:
%matplotlib qt
plt.scatter(spatial_data[:, 0], spatial_data[:, 1], c=spatial_data[:,2])
plt.colorbar()
plt.gca().set_aspect('equal')
plt.show()